In [1]:
#WPCM1 on Total Dead Kilometers Cost Matrix
#import timeit
#start = timeit.default_timer()
# All the program statements

from collections import defaultdict  # Defaultdict never raises a Key Error
import pandas as pd
import numpy as np
import random
import copy
import math

randomlist = []
demand = {}
random.seed(2) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    demand["D"+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in demand.items():
    sum = sum + v

supply = {}
for i in range(sum):
    supply[i] = 1.0

np.random.seed(2) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,20))).astype(float)

random.seed(2) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(2) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(2) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom
df_cost = df_tdkom.copy(deep=True)

#df_tdkom.columns = [("D"+str(i)) for i in range(1,4)]
df_result = pd.DataFrame(np.zeros(df_tdkom.shape))

In [2]:
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs = dk.to_dict()

#costs

In [3]:
g = {}
for x in supply:
    g[x] = sorted(costs[x].keys(), key=lambda g: costs[x][g], reverse=True)
for y in demand:
#     print(x)
    g[y] = sorted(costs.keys(), key=lambda g: costs[g][y] , reverse=True)

In [4]:

computed = {}

for x in supply:
    computed[x]={}
    for y in demand:
        computed[x][y]= - (costs[x][y] - costs[x][g[x][0]] - costs[g[y][0]][y])


In [5]:

max_computed = {}
for x in supply:
    max_computed[x] = sorted(computed[x].keys(), key=lambda g: computed[x][g], reverse=True)
# for y in demand:
# #     print(x)
#     max_computed[y] = sorted(computed.keys(), key=lambda g: computed[g][y], reverse=True)

#print(g)
#print(computed)
#print(max_computed)

res = dict((k, defaultdict(int)) for k in costs)


In [6]:
while g:
    
    old_d = "D"
    old_s = "S"
    
    ### Beginning of a loop
    # cleaning
    old_max = 0
    for x in supply:
        if computed[x][max_computed[x][0]] > old_max:
            #print(computed[x][max_computed[x][0]])
            old_max = computed[x][max_computed[x][0]]
            old_d = max_computed[x][0]
            old_s = x

    #print(old_max)
    #print(old_d)
    #print(old_s)

    # added to results matrix

    res[old_s][old_d] += 1

    demand[old_d] -= 1
    supply[old_s] -= 1

    
    ### Simultaneously ensures whether supply/demand gets cut
    if supply[old_s] == 0:
        for y in demand:
            if g[y][0] == old_s:
                for x in supply:
                    if len(g[y])>1:
                        computed[x][y] = computed[x][y] - costs[g[y][0]][y] + costs[g[y][1]][y]


    if demand[old_d] == 0:
        for x in supply:
            #go through all demand points within this supply and update computed cost
            if g[x][0] == old_d:
                for y in demand:
                    if len(g[x])>1:
                        computed[x][y] = computed[x][y] - costs[x][g[x][0]] + costs[x][g[x][1]]

    if supply[old_s] == 0:
        del g[old_s]
        del supply[old_s]
        del computed[old_s]
        #del max_computed[old_s]
        for k, n in demand.items():
            #if n == 0:
            g[k].remove(old_s)
            #max_computed[k].remove(old_s)

    if demand[old_d] == 0:
        del g[old_d]
        del demand[old_d]
        for k, n in supply.items():
            g[k].remove(old_d)
          # print("removing" + str(k) + "and" + str(old_d))
            #max_computed[k].remove(old_d)
            del computed[k][old_d]
          # print(max_computed[k])
        
    max_computed = {}
    for x in supply:
        max_computed[x] = sorted(computed[x].keys(), key=lambda g: computed[x][g], reverse=True)

In [7]:
cost = 0
#print(costs)
for x in costs.keys():
    for y in costs[x].keys():
        cost += costs[x][y] * res[x][y]

print("cost = " + str(cost))

cost = 255962.30970886414


stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))